In [1]:
pip install pmdarima

You should consider upgrading via the '/data/user/0/ru.iiec.pydroid3/files/aarch64-linux-android/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

In [3]:
df = pd.read_csv("DailyTemps.csv", parse_dates = ['DATE'], index_col = 'DATE')
df.head()

,MinTemp,MaxTemp,AvgTemp,Sunrise,Sunset
DATE,,,,,
2014-01-01,33.0,46.0,40.0,657,1756
2014-01-02,35.0,50.0,43.0,657,1756
2014-01-03,36.0,45.0,41.0,657,1757
2014-01-04,32.0,41.0,37.0,658,1757
2014-01-05,24.0,38.0,31.0,658,1758


In [4]:
df["AvgTemp"].plot() 

In [5]:
df = df.dropna()

In [6]:
df.isna().sum()

MinTemp    0
MaxTemp    0
AvgTemp    0
Sunrise    0
Sunset     0
dtype: int64

In [7]:
from statsmodels.tsa.stattools import adfuller

In [8]:
result = adfuller(df.AvgTemp)
p_val = result[1]
if p_val > 0.05:
    print("Non Stationary")
else:
    print("Stationary")

Stationary


In [9]:
from pmdarima import auto_arima

In [10]:
auto_model = auto_arima(df["AvgTemp"], trace = True)

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=8344.260, Time=7.23 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=10347.755, Time=0.10 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=8365.701, Time=0.38 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=9136.225, Time=0.75 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=19192.139, Time=0.06 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=8355.947, Time=4.12 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=8356.308, Time=7.42 sec
 ARIMA(3,0,2)(0,0,0)[0] intercept   : AIC=8347.317, Time=8.24 sec
 ARIMA(2,0,3)(0,0,0)[0] intercept   : AIC=8317.660, Time=8.33 sec
 ARIMA(1,0,3)(0,0,0)[0] intercept   : AIC=8330.191, Time=8.33 sec
 ARIMA(3,0,3)(0,0,0)[0] intercept   : AIC=8310.337, Time=10.33 sec
 ARIMA(4,0,3)(0,0,0)[0] intercept   : AIC=8332.504, Time=11.71 sec
 ARIMA(3,0,4)(0,0,0)[0] intercept   : AIC=8317.321, Time=11.36 sec
 ARIMA(2,0,4)(0,0,0)[0] intercept   : AIC=8305.753, Time=10.82 sec
 ARIMA(1,0,4)(0,0,0)[0] int

In [11]:
auto_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 1821
Model:               SARIMAX(1, 0, 5)   Log Likelihood               -4139.998
Date:                Fri, 09 Apr 2021   AIC                           8295.996
Time:                        16:47:22   BIC                           8340.053
Sample:                             0   HQIC                          8312.250
                               - 1821                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      1.3907      0.407      3.414      0.001       0.592       2.189
ar.L1          0.9701      0.009    110.620      0.000       0.953       0.987
ma.L1         -0.1189      0.024     -4.911      0.000      -0.166      -0.071
ma.L2         -0.2126      0.025     -8.645      0.000      -0.261      -0.164
ma.L3         -0.2034      0.024     -8.386      0.000      -0.251      -0.156
ma.L4         -0.1312      0.023     -5.705      0.000      -0.176      -0.086
ma.L5         -0.0428      0.025     -1.745      0.081      -0.091       0.005
sigma2         5.5373      0.175     31.710      0.000       5.195       5.880
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):                21.25
Prob(Q):                              0.92   Prob(JB):                         0.00
Heteroskedasticity (H):               0.81   Skew:                            -0.17
Prob(H) (two-sided):                  0.01   Kurtosis:                         3.40
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [12]:
x = df["AvgTemp"]

In [13]:
x.shape

(1821,)

In [14]:
train_len = int(0.8 * len(x))
train_len

1456

In [15]:
train = x[:train_len]
train.shape

(1456,)

In [16]:
test = x[train_len:]
test.shape

(365,)

In [17]:
# make a model 

from statsmodels.tsa.arima_model import ARIMA 
model = ARIMA(train, order =(1,0,5))

model_fit = model.fit()

model_fit.summary()

/data/user/0/ru.iiec.pydroid3/files/aarch64-linux-android/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/data/user/0/ru.iiec.pydroid3/files/aarch64-linux-android/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py

<class 'statsmodels.iolib.summary.Summary'>
"""
                              ARMA Model Results                              
==============================================================================
Dep. Variable:                AvgTemp   No. Observations:                 1456
Model:                     ARMA(1, 5)   Log Likelihood               -3334.671
Method:                       css-mle   S.D. of innovations              2.389
Date:                Fri, 09 Apr 2021   AIC                           6685.343
Time:                        16:47:25   BIC                           6727.610
Sample:                             0   HQIC                          6701.113
                                                                              
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            46.5293      0.860     54.102      0.000      44.844      48.215
ar.L1.AvgTemp     0.9844      0.009    112.835      0.000       0.967       1.002
ma.L1.AvgTemp    -0.1312      0.028     -4.704      0.000      -0.186      -0.077
ma.L2.AvgTemp    -0.2444      0.028     -8.852      0.000      -0.299      -0.190
ma.L3.AvgTemp    -0.2051      0.029     -7.029      0.000      -0.262      -0.148
ma.L4.AvgTemp    -0.1476      0.027     -5.486      0.000      -0.200      -0.095
ma.L5.AvgTemp    -0.0584      0.028     -2.082      0.037      -0.113      -0.003
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            1.0158           +0.0000j            1.0158            0.0000
MA.1            1.0910           -0.0000j            1.0910           -0.0000
MA.2            0.0718           -1.8558j            1.8572           -0.2438
MA.3            0.0718           +1.8558j            1.8572            0.2438
MA.4           -1.8803           -1.0060j            2.1325           -0.4218
MA.5           -1.8803           +1.0060j            2.1325            0.4218
-----------------------------------------------------------------------------
"""

In [18]:
plt.plot(train, label = "Training Data")
plt.plot(test, label = "Testing Data")
plt.legend()

In [19]:
y_pred_train = model_fit.predict()
y_pred_train

DATE
2014-01-01    46.529325
2014-01-02    41.196917
2014-01-03    43.977320
2014-01-04    41.710022
2014-01-05    38.275595
                ...    
2017-12-25    44.577266
2017-12-26    41.599461
2017-12-27    43.748000
2017-12-28    44.505478
2017-12-29    42.748242
Length: 1456, dtype: float64

In [20]:
# predictions on test data set 

# start -> len(train) & end -> len(train) - len(test) - 1 or len(x)-1

y_pred_test = model_fit.predict(start = len(train), end = len(x)-1)
                        
y_pred_test

/data/user/0/ru.iiec.pydroid3/files/aarch64-linux-android/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'


1456    42.929235
1457    42.907454
1458    43.061219
1459    43.165988
1460    43.203600
          ...    
1816    46.516803
1817    46.516998
1818    46.517190
1819    46.517378
1820    46.517564
Length: 365, dtype: float64

In [21]:
date_test = pd.date_range(start = '2017-12-30', end = '2018-12-29')
date_test

DatetimeIndex(['2017-12-30', '2017-12-31', '2018-01-01', '2018-01-02',
               '2018-01-03', '2018-01-04', '2018-01-05', '2018-01-06',
               '2018-01-07', '2018-01-08',
               ...
               '2018-12-20', '2018-12-21', '2018-12-22', '2018-12-23',
               '2018-12-24', '2018-12-25', '2018-12-26', '2018-12-27',
               '2018-12-28', '2018-12-29'],
              dtype='datetime64[ns]', length=365, freq='D')

In [22]:
y_pred_test.index = date_test
y_pred_test

2017-12-30    42.929235
2017-12-31    42.907454
2018-01-01    43.061219
2018-01-02    43.165988
2018-01-03    43.203600
                ...    
2018-12-25    46.516803
2018-12-26    46.516998
2018-12-27    46.517190
2018-12-28    46.517378
2018-12-29    46.517564
Freq: D, Length: 365, dtype: float64

In [23]:
test.plot()
y_pred_test.plot()

In [24]:
residuals = model_fit.resid
residuals.plot(kind="kde")

In [25]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(test, y_pred_test)

print("MSE is :", mse)
print("RMSE is :", np.sqrt(mse))

MSE is : 14.341740529961454
RMSE is : 3.787049053017594


In [26]:
# predict 30 future values

In [27]:
model = ARIMA(x , order = (1,0,5))
model_fit = model.fit()

/data/user/0/ru.iiec.pydroid3/files/aarch64-linux-android/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/data/user/0/ru.iiec.pydroid3/files/aarch64-linux-android/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py

In [28]:
forecast = model_fit.predict(start = len(x), end = len(x)+29)
forecast

/data/user/0/ru.iiec.pydroid3/files/aarch64-linux-android/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'


1821    46.418064
1822    46.113782
1823    45.617772
1824    45.249556
1825    45.116985
1826    45.136772
1827    45.156282
1828    45.175518
1829    45.194485
1830    45.213186
1831    45.231625
1832    45.249806
1833    45.267732
1834    45.285407
1835    45.302834
1836    45.320017
1837    45.336960
1838    45.353665
1839    45.370136
1840    45.386376
1841    45.402389
1842    45.418177
1843    45.433744
1844    45.449094
1845    45.464228
1846    45.479150
1847    45.493863
1848    45.508370
1849    45.522673
1850    45.536777
dtype: float64

In [29]:
forecast_date = pd.date_range(start = "2019-01-01", end = "2019-01-30")
forecast_date
print(len(forecast_date))
print(len(forecast))

30
30


In [30]:
forecast.index = forecast_date

In [31]:
forecast

2019-01-01    46.418064
2019-01-02    46.113782
2019-01-03    45.617772
2019-01-04    45.249556
2019-01-05    45.116985
2019-01-06    45.136772
2019-01-07    45.156282
2019-01-08    45.175518
2019-01-09    45.194485
2019-01-10    45.213186
2019-01-11    45.231625
2019-01-12    45.249806
2019-01-13    45.267732
2019-01-14    45.285407
2019-01-15    45.302834
2019-01-16    45.320017
2019-01-17    45.336960
2019-01-18    45.353665
2019-01-19    45.370136
2019-01-20    45.386376
2019-01-21    45.402389
2019-01-22    45.418177
2019-01-23    45.433744
2019-01-24    45.449094
2019-01-25    45.464228
2019-01-26    45.479150
2019-01-27    45.493863
2019-01-28    45.508370
2019-01-29    45.522673
2019-01-30    45.536777
Freq: D, dtype: float64

In [32]:
forecast.plot()

### Air-Line Passenger Data Set

In [33]:
data = pd.read_csv("international-airline-passengers.csv", parse_dates =["Month"], index_col = 'Month')
data.head()

,International airline passengers: monthly totals in thousands. Jan 49 ? Dec 60
Month,
1949-01,112.0
1949-02,118.0
1949-03,132.0
1949-04,129.0
1949-05,121.0


In [34]:
data = data.dropna()

In [35]:
data.isna().sum()

International airline passengers: monthly totals in thousands. Jan 49 ? Dec 60    0
dtype: int64

In [36]:
data.columns = ['MTIT'] #monthly total in thousands

In [37]:
data.head()

,MTIT
Month,
1949-01,112.0
1949-02,118.0
1949-03,132.0
1949-04,129.0
1949-05,121.0


In [38]:
#data["MTIT"].plot()

In [39]:
data.plot()

In [40]:
result = adfuller(data.MTIT)
p_val = result[1]
if p_val > 0.05:
    print("Non Stationary")
else:
    print("Stationary")

Non Stationary


In [41]:
# now we have to convert it in stationary

In [42]:
data["MTIT1"] = data["MTIT"]-data["MTIT"].shift(1)
data = data.dropna()
plt.plot(data.index , data.MTIT1)

In [43]:
result = adfuller(data.MTIT1)
p_val = result[1]
if p_val > 0.05:
    print("Non Stationary")
else:
    print("Stationary")

Non Stationary


In [44]:
# Seasonal Difference
 
data["MTIT2"] = data["MTIT"]-data["MTIT"].shift(7)
data = data.dropna()
plt.plot(data.index, data.MTIT2)

<ipython-input-44-9b60a6188b61>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["MTIT2"] = data["MTIT"]-data["MTIT"].shift(7)


In [45]:
result = adfuller(data.MTIT2)
p_val = result[1]
if p_val > 0.05:
    print("Non Stationary")
else:
    print("Stationary")

Stationary


In [46]:
auto_model = auto_arima(data["MTIT2"], trace = True)

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=1333.946, Time=2.11 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=1564.277, Time=0.03 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=1434.725, Time=0.09 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=1433.530, Time=0.50 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=1570.988, Time=0.02 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=1376.357, Time=1.10 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=1334.436, Time=0.77 sec
 ARIMA(3,0,2)(0,0,0)[0] intercept   : AIC=inf, Time=2.01 sec
 ARIMA(2,0,3)(0,0,0)[0] intercept   : AIC=inf, Time=2.45 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=1378.380, Time=0.66 sec
 ARIMA(1,0,3)(0,0,0)[0] intercept   : AIC=inf, Time=1.71 sec
 ARIMA(3,0,1)(0,0,0)[0] intercept   : AIC=1366.350, Time=1.58 sec
 ARIMA(3,0,3)(0,0,0)[0] intercept   : AIC=inf, Time=2.35 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=1356.422, Time=0.61 sec

Best model:  ARIMA(2,0,2)(0,0,0)[0] intercept
Total 

In [47]:
auto_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  136
Model:               SARIMAX(2, 0, 2)   Log Likelihood                -660.973
Date:                Fri, 09 Apr 2021   AIC                           1333.946
Time:                        16:47:51   BIC                           1351.422
Sample:                             0   HQIC                          1341.048
                                - 136                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      4.7374      0.731      6.478      0.000       3.304       6.171
ar.L1          1.5764      0.056     28.371      0.000       1.468       1.685
ar.L2         -0.8242      0.047    -17.472      0.000      -0.917      -0.732
ma.L1         -0.4354      0.093     -4.686      0.000      -0.618      -0.253
ma.L2         -0.3982      0.082     -4.860      0.000      -0.559      -0.238
sigma2       952.0422    114.398      8.322      0.000     727.826    1176.258
===================================================================================
Ljung-Box (L1) (Q):                   1.57   Jarque-Bera (JB):                 1.13
Prob(Q):                              0.21   Prob(JB):                         0.57
Heteroskedasticity (H):               4.36   Skew:                             0.21
Prob(H) (two-sided):                  0.00   Kurtosis:                         3.15
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [48]:
x = data.MTIT2

In [49]:
x.shape

(136,)

In [50]:
train_len = int(0.8 * len(x))
train_len

108

In [51]:
train = x[:train_len]
train.shape

(108,)

In [52]:
test = x[train_len:]
test.shape

(28,)

In [53]:
# make a model 

from statsmodels.tsa.arima_model import ARIMA
 
model = ARIMA(train, order =(2,0,2))

model_fit = model.fit()

model_fit.summary()

/data/user/0/ru.iiec.pydroid3/files/aarch64-linux-android/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/data/user/0/ru.iiec.pydroid3/files/aarch64-linux-android/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py

<class 'statsmodels.iolib.summary.Summary'>
"""
                              ARMA Model Results                              
==============================================================================
Dep. Variable:                  MTIT2   No. Observations:                  108
Model:                     ARMA(2, 2)   Log Likelihood                -519.448
Method:                       css-mle   S.D. of innovations        1670663.896
Date:                Fri, 09 Apr 2021   AIC                           1050.896
Time:                        16:47:52   BIC                           1066.989
Sample:                    09-01-1949   HQIC                          1057.421
                         - 08-01-1958                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          17.2215        nan        nan        nan         nan         nan
ar.L1.MTIT2     1.7409        nan        nan        nan         nan         nan
ar.L2.MTIT2    -1.0000        nan        nan        nan         nan         nan
ma.L1.MTIT2    -1.4772      0.104    -14.222      0.000      -1.681      -1.274
ma.L2.MTIT2     0.6815      0.135      5.063      0.000       0.418       0.945
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            0.8704           -0.4923j            1.0000           -0.0819
AR.2            0.8704           +0.4923j            1.0000            0.0819
MA.1            1.0838           -0.5411j            1.2113           -0.0737
MA.2            1.0838           +0.5411j            1.2113            0.0737
-----------------------------------------------------------------------------
"""

In [54]:
plt.plot(train, label = "Training Data")
plt.plot(test, label = "Testing Data")
plt.legend()

In [55]:
y_pred_train = model_fit.predict()
y_pred_train

1949-09-01     17.221526
1949-10-01     17.899139
1949-11-01    -36.168664
1949-12-01    -33.671893
1950-01-01     -5.383011
                 ...    
1958-04-01    -24.821127
1958-05-01     35.523392
1958-06-01     95.814677
1958-07-01    151.177811
1958-08-01    161.951727
Freq: MS, Length: 108, dtype: float64

In [56]:
# predictions on test data set 
# start -> len(train) & end -> len(train) - len(test) - 1 or len(x)-1
y_pred_test = model_fit.predict(start = len(train), end = len(x)-1)
                        
y_pred_test

1958-09-01    134.808794
1958-10-01     76.225253
1958-11-01      2.352356
1958-12-01    -67.667561
1959-01-01   -115.690564
1959-02-01   -129.272678
1959-03-01   -104.894436
1959-04-01    -48.872859
1959-05-01     24.275445
1959-06-01     95.595897
1959-07-01    146.607563
1959-08-01    164.092031
1959-09-01    143.518634
1959-10-01     90.218460
1959-11-01     18.002935
1959-12-01    -54.415070
1960-01-01   -108.270217
1960-02-01   -129.607275
1960-03-01   -112.897274
1960-04-01    -62.470199
1960-05-01      8.607017
1960-06-01     81.916468
1960-07-01    138.461820
1960-08-01    163.590741
1960-09-01    150.791693
1960-10-01    103.381235
1960-11-01     33.644618
1960-12-01    -40.347636
Freq: MS, dtype: float64

In [57]:
test.plot()
y_pred_test.plot()

In [58]:
residuals = model_fit.resid
residuals.plot(kind="kde")

In [59]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(test, y_pred_test)

print("MSE is :", mse)

print("RMSE is :", np.sqrt(mse))

MSE is : 2919.691891335386
RMSE is : 54.034173365893054


In [60]:
# predict 30 future value

In [61]:
model = ARIMA(x , order = (2,0,2))
model_fit = model.fit()

/data/user/0/ru.iiec.pydroid3/files/aarch64-linux-android/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/data/user/0/ru.iiec.pydroid3/files/aarch64-linux-android/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py

In [62]:
forecast = model_fit.predict(start = len(x), end = len(x)+29)
forecast

1961-01-01   -29.752019
1961-02-01   -47.951698
1961-03-01   -46.332759
1961-04-01   -28.779838
1961-05-01    -2.443148
1961-06-01    24.607296
1961-07-01    45.543059
1961-08-01    56.251113
1961-09-01    55.875750
1961-10-01    46.458087
1961-11-01    31.921122
1961-12-01    16.766857
1962-01-01     4.858954
1962-02-01    -1.422467
1962-03-01    -1.509850
1962-04-01     3.529747
1962-05-01    11.546377
1962-06-01    20.030290
1962-07-01    26.797077
1962-08-01    30.471772
1962-09-01    30.687292
1962-10-01    27.998242
1962-11-01    23.581487
1962-12-01    18.835153
1963-01-01    14.993289
1963-02-01    12.848917
1963-03-01    12.635038
1963-04-01    14.065332
1963-05-01    16.496387
1963-06-01    19.149897
Freq: MS, dtype: float64

In [63]:
forecast.plot()